# New Fitness Functions for the EvoGFuzzer
This is the Jupyter Notebook for the paper "New Fitness Functions for the EvoGFuzzer".
It builds on the EvoGFuzz Project by **Martin Eberlein et. al.**.

This Notebook solemnly contains our additions to this project, namely the changes to the fuzz() function and the improved (and one additional) fitness function.

This Notebook picks up from the paper at section 4 and focusses on the implementation of the code relevant for sections 4 and 5.

The introduction as well as explanations to related work, our contribution, as well as a short discourse about the fitness function and the grammer can be found in the paper in the sections 1 to 3. The notebook closes with a quick remark to section 6 and 7, which can be read in detail in our paper as well.


# Evaluating different fitness functions for EvoGFuzz

In our project we implement the three given fitness functions with a ```naive```, an ```improved``` and a ```sophisticated``` approach, that were given in the **EvoGFuzz** paper. We then came up with a new approach that uses and aims to improve the ```sophisticated``` approach. We call it the ```ratioed sophisticated``` approach. In this notebook we evaluate each of the approach.


We use the same example as **EvoGFuzz** and therefore need to define our calculator, its oracle and the grammar, as well as covering most imports we need later on and our Benchmark program.

In [58]:
import math

def calculator(inp: str) -> float:
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

In [ ]:
from typing import List

from debugging_framework.benchmark.program import BenchmarkProgram

from tests4py.api.logging import deactivate, debug
deactivate()

from debugging_benchmark.tests4py_benchmark.repository import (
    PysnooperBenchmarkRepository,
    CookieCutterBenchmarkRepository
)

repos = [
    PysnooperBenchmarkRepository(),
    CookieCutterBenchmarkRepository(),
]

programs: List[BenchmarkProgram] = []
List[BenchmarkProgram]
for repo in repos:
    tmp: List[BenchmarkProgram] = repo.build()
    for prog in tmp:
        programs.append(prog)

PYSNOOPER_2: Grammar = programs[0].get_grammar()
PYSNOOPER_3: Grammar = programs[1].get_grammar()
COOKIECUTTER_2: Grammar = programs[2].get_grammar()
COOKIECUTTER_3: Grammar = programs[3].get_grammar()
COOKIECUTTER_4: Grammar = programs[4].get_grammar()


In [59]:
# Make sure you use the OracleResult from the debugging_framework library
from debugging_framework.input.oracle import OracleResult

def oracle(inp: str):
    try:
        calculator(inp)
    except Exception as exc:
        return OracleResult.FAILING
    
    return OracleResult.PASSING

In [60]:
from debugging_framework.types import Grammar
from debugging_framework.fuzzingbook.grammar import is_valid_grammar

CALC_GRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
assert is_valid_grammar(CALC_GRAMMAR)

In [61]:
EXPR_GRAMMAR:  Grammar = {
    "<start>":
        ["<expr>"],

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],

    "<factor>":
        ["+<factor>",
         "-<factor>",
         "(<expr>)",
         "<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
assert is_valid_grammar(EXPR_GRAMMAR)

In [62]:
COMPL_CALC_GRAMMAR: Grammar = {
    "<start>":
        ["<expr>"],

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>", "<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],

    "<factor>":
        ["+<factor>",
         "-<factor>",
         "(<expr>)",
         "<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
assert is_valid_grammar(COMPL_CALC_GRAMMAR)

In [ ]:
assert is_valid_grammar(PYSNOOPER_2)
assert is_valid_grammar(PYSNOOPER_3)
assert is_valid_grammar(COOKIECUTTER_2)
assert is_valid_grammar(COOKIECUTTER_3)
assert is_valid_grammar(COOKIECUTTER_4)

For the new fitness functions we also need to define some helper functions, that we use later on. 

We start by defining a function ```count_expansions``` for the ```improved``` approach, that returns the number of expansions of a tree, when given the first children of the root of the derivation tree.

As for our ```ratioed sophisticated``` approach we need the maximum height of the tree to devide by it, that also returns the max height of the derivation tree. We named it ```calculate_height_and_degrees```. Its also being used for the normal ```sophisticated_fitness_function```.

Lastly we need to count the expansions for the ```diff_expansions_fitness_function``` (which is explained later), for which ```get_diff_expansions``` is being used.

In [63]:
def count_expansions(children):
    """
   Calculate the number of expansions of the Derivation Trees.
   :param children: A List of Derivation Trees
   :return: The total number of expansions of all children.
    """
    if children == []:
        return 0

    counter = 1
    for child in children:
        _, next_children = child
        counter += count_expansions(next_children)

    return counter

In [65]:
def calculate_height_and_degrees(children, height):
    """
   Calculate the maximal height and the sum of the degrees of the Derivation Trees.
   :param children: A List of Derivation Trees
   :param height: The current height of the Sub Derivation Tree.
   :return: The maximal height and the sum of the degrees.
    """
    max_height = height
    degrees = 0

    for child in children:
        _, next_children = child
        degrees += len(next_children) ** height
        next_score, next_height = calculate_height_and_degrees(next_children, height + 1)
        degrees += next_score
        if next_height > max_height:
            max_height = next_height

    return max_height, degrees

In [66]:
def get_diff_expansions(children, exp_set):
    """
   Calculate a set of used production rules of Derivation Trees.
   :param children: A List of Derivation Trees
   :param exp_set: The current set of seen expansions.
   :return: The set of all used production rules.
    """
    expansion = ""
    for child in children:
        node, _ = child
        expansion += node

    exp_set.add(expansion)

    for child in children:
        _, next_children = child
        exp_set.update(get_diff_expansions(next_children, exp_set))

    return exp_set


As mentioned before the **EvoGFuzz** paper took a very simple approach for calculating the fitness of each input. The paper itself suggested three different functions that might improve the outcome for future work. A ```naive```, an ```improved``` and a ```sophisticated``` approach. 

# 4 Solution Description

The **Standard Fitness Function** is a binary function that assigns the value of 1 if an exception is found and 0 otherwise. It does not reward more interesting and complex exception types. In
order to reward more interesting inputs we implement continuous functions that will have the same structure. We calculate the fitness of an input as followed:

$$
fitness(x) = score_{feedback}(x) + score_{structure}(x)
$$

The feedback score is 0 if no exception is triggered. If an exception is triggered we set the feedback score to an integer value depending on the structure score of each fitness function so that an exception triggering input has more weight. The goal is to maximize the fitness score.
First we are going to implement the fitness functions as mentioned in Eberlein’s bachelor thesis.

The ```naive``` approach simply takes in the length of the input. To reward a failing input, we add 100 to the score. We implement this in the ```naive_fitness_function```.

For the ```improved``` approach we count the expansions of the derivation tree, square it and devide it by the length of the input and multiply the length by a paramater $\lambda$. 

Instead of ranking inputs just because of their length, the improved fitness function counts the number of expansions. The hope is that with more expansions used more uncommon exceptions will be found. 
Because we want to reward long inputs that were produced by many expansions instead of inputs that just produce long words, we set the number of expansions in relation to the length of the input.

For the **improved fitness function** we calculate the structure score as followed:

$$
score_{structure}(x) = \frac{\mu(x)^2}{\lambda \cdot length(x)} \text{.}
$$

We reward failing inputs as before. This is implemented in the ```improved_fitness_function```.

The ```sophisticated``` approach rewards more complex expansions. For that we also need the degree of each node, which is then exponantiated by the height of it. As the fitness scores tends to get quite large, we need to reward failing inputs by adding $2^{100}$ to it. We want expansions that have a high fan out and appear deep inside the derivation tree.

$$
score_{structure}(x) = \sum_{\forall v\in V}deg(v)^{h(v)} \text{.}
$$

This is implemented in the ```sophisticated_fitness_funtion```.

For our own approach, because we want to reward complex structures that have a high fan out deep inside the tree and not just trees that are very long, we decided to put the sophisticated fitness function in relation to the overall height of the derivation tree.. We calculate our score as follows:
$$
score_{structure}(x) = \frac{\sum_{\forall v\in V}deg(v)^{h(v)}}{\lambda \cdot h} \text{.}
$$
We reward a failing input the same as in the first 2 approaches. It's implemented in the ```ratio_sophisticated_fitness_function```.

In [67]:
from debugging_framework.input.oracle import OracleResult

from evogfuzz.input import Input
from evogfuzz.helper_functions import calculate_height_and_degrees, count_expansions, get_diff_expansions


def fitness_function_failure(test_input: Input) -> float:
    return get_fitness(test_input)


def get_fitness(test_input: Input) -> int:
    return standard_fitness_function(test_input)


def standard_fitness_function(test_input: Input) -> int:
    if test_input.oracle == OracleResult.FAILING:
        return 1
    else:
        return 0

The fitness functions themselves are being implemented as follows:

In [68]:
def naive_fitness_function(test_input: Input) -> int:
    """
    The naive approach takes in the length of the input. To reward a failing input, we add 100 to the score.
    :param test_input: Input.
    :return: The fitness of the input.
    """
    score_structure = len(str(test_input))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [69]:
def improved_fitness_function(test_input: Input) -> float:
    """
    The improved fitness function puts the length into perspective to the number of expansions.
    :param test_input: Input.
    :return: The fitness of the input.
    """
    _, children = test_input.tree
    number_expansions = count_expansions(children)
    lam = 100
    if len(str(test_input)) == 0:
        score_structure = 0
    else:
        score_structure = (number_expansions ** 2) / (lam * len(str(test_input)))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [70]:
def sophisticated_fitness_function(test_input: Input) -> int:
    """
    The sophisticated fitness function considers the more complex expansions.
    An expansion is more complex if it is more nested.
    :param test_input: Input.
    :return: The fitness of the input.
    """
    _, score_structure = calculate_height_and_degrees([test_input.tree], 0)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 2 ** 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [71]:
def ratio_sophisticated_fitness_function(test_input: Input) -> float:
    """
    The ratioed sophisticated fitness function puts the complexity into perspective to the height of the Derivation Tree.
    :param test_input: Input.
    :return: The fitness of the input.
    """
    lam = 2 ** 50
    height, degrees = calculate_height_and_degrees([test_input.tree], 0)
    if height == 0:
        score_structure = 0
    else:
        score_structure = degrees / (lam * height)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [72]:
def diff_expansions_fitness_function(test_input: Input) -> int:
    """
    The different expansions fitness function favors inputs with a higher variation of used production ruleste.
    :param test_input: Input.
    :return: The fitness of the input.
    """
    score_structure = len(get_diff_expansions([test_input.tree], set()))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

The **sophisticated fitness function** still rewards inputs, that use the same expansion rule many times. This is why we developed another fitness function, that counts the number of different rules applied in the derivation tree. In the hope that inputs that use as many rules as possible are more complex than ones that only use some of the rules.

$$
score_{structure}(x) = \delta
$$

where $\delta$ is the number of different rules used.

# 5 Implementation
For our project we forked Eberlein's https://github.com/martineberlein/evogfuzzplusplusm. Our work can be found at https://github.com/lukastriescoding/evogfuzzplusplus.

The ground structure of EvoGFuzz stays the same. There are only two parts of the EvoGFuzz implementation that we changed. One is the ***’fuzz()’*** function, which now returns the bugs found after each iteration. This is important as we need that data for ***RQ2***. We also amended the ’fitness function’ module with our five new fitness functions. To implement them one needs to import the intended function from said module and set the given parameter when instantiating a new EvoGFuzz instance.
We created the ***’eval’*** directory, where our script for the evaluation is placed. To start the evaluation, one can change the parameters in the main function. Possible changes include:

- Number of trials
- Grammar on which the evaluation operates
- Initial inputs that belong to the grammar
- Iteration

To call the logic, the **eval.py** needs to be executed. It produces an output file, that contains the results. To differentiate multiple output files, the program takes an argument, which will be concatenated to the file-name. The call **python eval.py calc\_grammer\_1** will produce the output-file *"resultscalc\_grammar\_1.txt"*. This file contains a bunch of lists containing the results. Each list entry represents a trial and each list contains all the trials for one iteration. The file contains all the results for all of the fitness functions which are seperated by a new line each.

In [ ]:
import sys

from preprocess import programs, oracle, CALC_GRAMMAR, EXPR_GRAMMAR, COMPL_CALC_GRAMMAR, PYSNOOPER_2, PYSNOOPER_3, COOKIECUTTER_2, COOKIECUTTER_3, COOKIECUTTER_4

from evogfuzz.evogfuzz_class import EvoGFuzz
from evogfuzz.fitness_functions import naive_fitness_function, improved_fitness_function, sophisticated_fitness_function, ratio_sophisticated_fitness_function, diff_expansions_fitness_function


def write_output(file, dict_found_exc_inp):
    for iteration in dict_found_exc_inp.keys():
        file.write(f"{dict_found_exc_inp[iteration]}\n")
    file.write("\n")

In [ ]:
def eval_fitness(trials, grammar, oracle, initial_inputs, iterations):
    """
    Evaluate the different fitness functions.
    :param trials: The number of iterations we use to calculate the found exception inputs.
    :param initial_inputs: The input from which EvoGFuzz starts to train.
    :param iterations: The number of iterations EvoGFuzz trains.
    :return: The total number of found exception inputs per iteration.
    """

    dict_stand = {i: [] for i in range(iterations)}
    dict_naive = {i: [] for i in range(iterations)}
    dict_impr = {i: [] for i in range(iterations)}
    dict_soph = {i: [] for i in range(iterations)}
    dict_ratio_soph = {i: [] for i in range(iterations)}
    dict_diff_exp = {i: [] for i in range(iterations)}

    dicts_list = []

    for i in range(trials):
        epp_stand = EvoGFuzz(
            grammar=grammar, oracle=oracle, inputs=initial_inputs, iterations=iterations
        )

        epp_naive = EvoGFuzz(
            grammar=grammar, oracle=oracle, inputs=initial_inputs, iterations=iterations,
            fitness_function=naive_fitness_function
        )

        epp_impr = EvoGFuzz(
            grammar=grammar, oracle=oracle, inputs=initial_inputs, iterations=iterations,
            fitness_function=improved_fitness_function
        )

        epp_soph = EvoGFuzz(
            grammar=grammar, oracle=oracle, inputs=initial_inputs, iterations=iterations,
            fitness_function=sophisticated_fitness_function
        )

        epp_ratio_soph = EvoGFuzz(
            grammar=grammar, oracle=oracle, inputs=initial_inputs, iterations=iterations,
            fitness_function=ratio_sophisticated_fitness_function
        )

        epp_diff_exp = EvoGFuzz(
            grammar=grammar, oracle=oracle, inputs=initial_inputs, iterations=iterations,
            fitness_function=diff_expansions_fitness_function
        )

        found_exc_inp_stand = epp_stand.fuzz()
        for iteration in found_exc_inp_stand.keys():
            dict_stand[iteration] += [len(found_exc_inp_stand[iteration])]
        dicts_list.append(dict_stand)

        found_exc_inp_naive = epp_naive.fuzz()
        for iteration in found_exc_inp_naive.keys():
            dict_naive[iteration] += [len(found_exc_inp_naive[iteration])]
        dicts_list.append(dict_naive)

        found_exc_inp_impr = epp_impr.fuzz()
        for iteration in found_exc_inp_impr.keys():
            dict_impr[iteration] += [len(found_exc_inp_impr[iteration])]
        dicts_list.append(dict_impr)

        found_exc_inp_soph = epp_soph.fuzz()
        for iteration in found_exc_inp_soph.keys():
            dict_soph[iteration] += [len(found_exc_inp_soph[iteration])]
        dicts_list.append(dict_soph)

        found_exc_inp_ratio_soph = epp_ratio_soph.fuzz()
        for iteration in found_exc_inp_ratio_soph.keys():
            dict_ratio_soph[iteration] += [len(found_exc_inp_ratio_soph[iteration])]
        dicts_list.append(dict_ratio_soph)

        found_exc_inp_diff_exp = epp_diff_exp.fuzz()
        for iteration in found_exc_inp_diff_exp.keys():
            dict_diff_exp[iteration] += [len(found_exc_inp_diff_exp[iteration])]
        dicts_list.append(dict_diff_exp)

    return dicts_list

In [ ]:
def main(trials, grammar, oracle, initial_inputs, iterations):
    input = sys.argv[1]
    filename = "results_" + input + ".txt"

    dicts_list = eval_fitness(trials, grammar, oracle, initial_inputs, iterations)

    with open(filename, "w") as file:
        write_output(file, dicts_list[0])
        write_output(file, dicts_list[1])
        write_output(file, dicts_list[2])
        write_output(file, dicts_list[3])
        write_output(file, dicts_list[4])
        write_output(file, dicts_list[5])


if __name__ == "__main__":
    grammars = {'PYSNOOPER_2': [PYSNOOPER_2, programs[0].get_oracle(), programs[0].get_initial_inputs()],
                'PYSNOOPER_3': [PYSNOOPER_3, programs[1].get_oracle(), programs[1].get_initial_inputs()],
                'COOKIECUTTER_2': [COOKIECUTTER_2, programs[2].get_oracle(), programs[2].get_initial_inputs()],
                'COOKIECUTTER_3': [COOKIECUTTER_3, programs[3].get_oracle(), programs[3].get_initial_inputs()],
                'COOKIECUTTER_4': [COOKIECUTTER_4, programs[4].get_oracle(), programs[4].get_initial_inputs()],
                'CALC_GRAMMAR': [CALC_GRAMMAR, oracle, ['sqrt(1)', 'cos(912)', 'tan(4)']],
                'EXPR_GRAMMAR': [EXPR_GRAMMAR, oracle, ['2 + 2', '-6 / 9', '-(23 * 7)']],
                'COMPL_CALC_GRAMMAR': [COMPL_CALC_GRAMMAR, oracle, ['sqrt(1)', 'cos(912)', 'tan(4)']]}

    trials = 30
    chosen_grammar = 'PYSNOOPER_2'
    grammar = grammars[chosen_grammar][0]
    oracle = grammars[chosen_grammar][1]
    initial_inputs = grammars[chosen_grammar][2]
    iterations = 10

    main(trials, grammar, oracle, initial_inputs, iterations)


To evaluate the now produced data we also added a ```mannwhitneyu test.py``` that takes one of these files and directly produces the content of a LATEX-table that can be seen later on in the
paper. It uses functions defined in ```input_reader.py```.

**input_Reader.py:**

In [ ]:
def get_lists(fitfunc: str, iteration: int, filename: str):
    with open(filename, 'r') as file:
        lines = file.readlines()

    if fitfunc == 'stand':
        our_list = lines[iteration][1:-2].split(', ')
    elif fitfunc == 'naive':
        our_list = lines[iteration + 11][1:-2].split(', ')
    elif fitfunc == 'impr':
        our_list = lines[iteration + 22][1:-2].split(', ')
    elif fitfunc == 'soph':
        our_list = lines[iteration + 33][1:-2].split(', ')
    elif fitfunc == 'ratio_soph':
        our_list = lines[iteration + 44][1:-2].split(', ')
    elif fitfunc == 'diff_exp':
        our_list = lines[iteration + 55][1:-2].split(', ')
    else:
        raise Exception("fitness_function not found")

    our_list_int = []
    for value in our_list:
        our_list_int.append(int(value))

    return our_list_int


**mannwhitneyu test.py**

In [ ]:
from scipy import stats
from input_reader import get_lists


def mannwhitneyu_test(stand, other):
    _, p_val = stats.mannwhitneyu(stand, other)
    return p_val


def main():
    filename = "data/resultscompl.txt"
    iterations = 10

    for i in range(iterations):
        stand = get_lists("stand", i, filename)
        naive = get_lists("naive", i, filename)
        impr = get_lists("impr", i, filename)
        soph = get_lists("soph", i, filename)
        ratio_soph = get_lists("ratio_soph", i, filename)
        diff_exp = get_lists("diff_exp", i, filename)

        p_val_naive = mannwhitneyu_test(diff_exp, naive)
        p_val_impr = mannwhitneyu_test(diff_exp, impr)
        p_val_soph = mannwhitneyu_test(diff_exp, soph)
        p_val_stand = mannwhitneyu_test(diff_exp, stand)
        p_val_ratio_soph = mannwhitneyu_test(diff_exp, ratio_soph)

        cur_fitfunc = diff_exp
        if p_val_naive <= 0.05:
            if sum(cur_fitfunc) < sum(naive):
                p_val_naive = '\color{red}$\ding{55}$\color{black}'  #cross
            else:
                p_val_naive = '\color{red}$\ding{51}$\color{black}'  #checkmark
        else:
            p_val_naive = round(p_val_naive, 2)

        if p_val_impr <= 0.05:
            if sum(cur_fitfunc) < sum(impr):
                p_val_impr = '\color{red}$\ding{55}$\color{black}'  # cross
            else:
                p_val_impr = '\color{red}$\ding{51}$\color{black}'  # checkmark
        else:
            p_val_impr = round(p_val_impr, 2)

        if p_val_soph <= 0.05:
            if sum(cur_fitfunc) < sum(soph):
                p_val_soph = '\color{red}$\ding{55}$\color{black}'  # cross
            else:
                p_val_soph = '\color{red}$\ding{51}$\color{black}'  # checkmark
        else:
            p_val_soph = round(p_val_soph, 2)

        if p_val_stand <= 0.05:
            if sum(cur_fitfunc) < sum(stand):
                p_val_stand = '\color{red}$\ding{55}$\color{black}'  # cross
            else:
                p_val_stand = '\color{red}$\ding{51}$\color{black}'  # checkmark
        else:
            p_val_stand = round(p_val_stand, 2)

        if p_val_ratio_soph <= 0.05:
            if sum(cur_fitfunc) < sum(ratio_soph):
                p_val_ratio_soph = '\color{red}$\ding{55}$\color{black}'  # cross
            else:
                p_val_ratio_soph = '\color{red}$\ding{51}$\color{black}'  # checkmark
        else:
            p_val_ratio_soph = round(p_val_ratio_soph, 2)

        print(i, '&', p_val_stand, '&', p_val_naive, '&', p_val_impr, '&', p_val_soph, '&', p_val_ratio_soph, "\\\\")


if __name__ == "__main__":
    main()


# 6 Evaluation
All the Data generated by us is visible in the ```data``` Folder.
In this section we want to evaluate our fitness function, compare it to the standard fitness function that was implemented before and ask the following research questions:

- **RQ1** How does the implementation of a new fitness function in EvoGFuzz impact the number of exceptions triggered?
- **RQ2** How does the implementation of a new fitness function in EvoGFuzz impact the number of exceptions triggered in each iteration?

To answer these questions, we conducted a **Mann-Whitney-U test**.

All our Results can be found in this section of the paper, as well as an explanation of our setup.

The **conclusions**, **visualization** by tables as well as a **discussion** can be found in the next section
##### **7 Discussion and Conclusion**
in the paper itself as well.

# This concludes this Notebook.